### 1. Sequential API 사용하여 model 만들기.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_boston
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
warnings.filterwarnings('ignore')                              # Turn the warnings off.
%matplotlib inline

#### 1.1. 데이터 읽어 오기:

In [ ]:
# 내장 데이터 불러오기.
data_raw = load_boston()
X = data_raw['data']
y = data_raw['target'].reshape(-1,1)
header = data_raw['feature_names']

In [ ]:
data_raw.keys()

In [ ]:
# 보기.
df = pd.DataFrame(X, columns = data_raw['feature_names'])
df.head()

In [ ]:
# 데이터 쪼개기.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
n_vars = X_train.shape[1]

In [ ]:
# X를 스케일링 한다.
# X를 스케일링 해주어야 학습이 잘 된다!
my_scaler = MinMaxScaler()
my_scaler.fit(X_train)                           # X_train만 가지고 스케일러 학습 실행!
X_train = my_scaler.transform(X_train)
X_test = my_scaler.transform(X_test)

#### 1.2. Sequential API model 정의:

In [ ]:
# Sequential object 객체에 layer 추가.
my_model1 = Sequential()
my_model1.add(Dense(input_dim = n_vars, units = 1, activation="linear"))    # 선형회귀의 유일한 한개의 층.

In [ ]:
# 모델의 summary (내부 구조) 출력.
# 파라미터의 수 = n_vars + 1 = 13 + 1 = 14.
my_model1.summary()

#### 1.3. Hyperparameter 와 Optimizer 정의:

In [ ]:
# Hyperparameters.
n_epochs = 700
batch_size = 32
learn_rate = 0.005

In [ ]:
# Optimizer 객체 생성 후 컴파일한다.
my_optimizer=Adam(lr=learn_rate)
my_model1.compile(loss = "mse", optimizer = my_optimizer, metrics=["mse"])

#### 1.4. 학습 & History 시각화:

In [ ]:
# 학습.
# verbose = 0 : 출력 없음. verbose = 1 : 출력 함.
my_summary = my_model1.fit(X_train, y_train, epochs=n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 0)

In [ ]:
# 학습된 파라미터 출력.
print("Weights : ", my_model1.layers[0].get_weights()[0].reshape(1,-1))
print("Bias    : ", my_model1.layers[0].get_weights()[1])

In [ ]:
# 학습 history의 key 확인.
my_summary.history.keys()

In [ ]:
# 학습 history 출력. 
n_skip = 10                            # Skip the first few steps.
plt.plot(my_summary.history['loss'][n_skip:], c="b")
plt.plot(my_summary.history['val_loss'][n_skip:], c="g")
plt.title('Training History')
plt.ylabel('LOSS')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

#### 1.5. 테스팅: 

In [ ]:
# 예측 및 평가.
y_pred = my_model1.predict(X_test)
RMSE = np.sqrt(np.mean((y_test-y_pred)**2))
np.round(RMSE,3)

In [ ]:
# evaluate() 메서드를 사용해서 평가.
MSE = my_model1.evaluate(X_test, y_test, verbose=0)[1]        # 0 = loss value. 1 = metric value.
RMSE = np.sqrt(MSE)
print("Test RMSE : {}".format(np.round(RMSE,3)))

In [ ]:
# 새로운 조건이 주어졌을 때 예측.
X_new = np.array([[0.03, 0.0, 13.0, 0.0, 0.4, 4.3, 23.5, 1.9, 1.0, 273.0, 18.0, 380.0, 7.5]])  # A row array.
X_new = my_scaler.transform(X_new)                                                             # 스케일링 적용 필수!!!
# Y_pred_new = my_model1.predict(X_new)
Y_pred_new = my_model1.predict(X_new)
print(np.round(Y_pred_new[0,0],3))

### 2. Functional API 사용하여 model 만들기.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

#### 2.1. Functional API model 정의:

In [ ]:
my_input = Input(shape=(n_vars,))                                     # Input layer.
my_output = Dense(units=1,activation='linear')(my_input)              # Output layer.

In [ ]:
my_model2 = Model(inputs=my_input,outputs=my_output)                   # The model.

In [ ]:
# Summary of the model.
# 파라미터의 수 = n_vars + 1 = 13 + 1 = 14.
my_model2.summary()

In [ ]:
# Optimizer 객체 생성 후 컴파일한다.
my_optimizer=Adam(lr=learn_rate)
my_model2.compile(loss = "mse", optimizer = my_optimizer, metrics=["mse"])    # Loss = MAE (L1) and Metrics = MSE (L2). 

#### 2.2. 학습 & History 시각화:

In [ ]:
# 학습.
# verbose = 0 : 출력 없음. verbose = 1 : 출력 함.
my_summary = my_model2.fit(X_train, y_train, epochs=n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 0)

In [ ]:
# 학습된 파라미터 출력.
# 스케일링된 X_train을 사용해서 계산했기 때문에 직관적인 해석은 어렵다!
print("Weights : ", my_model2.layers[1].get_weights()[0].reshape(1,-1))
print("Bias    : ", my_model2.layers[1].get_weights()[1])

In [ ]:
# 학습 history 출력. 
n_skip = 10                 # Skip the first few steps.
plt.plot(my_summary.history['mse'][n_skip:], c="b")
plt.plot(my_summary.history['val_mse'][n_skip:], c="g")
plt.title('Training History')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

#### 2.3. 테스팅: 

In [ ]:
# evaluate() 메서드를 사용해서 평가.
MSE = my_model2.evaluate(X_test, y_test, verbose=0)[1]        # 0 = loss value, 1 = metrics value 반환.
RMSE = np.sqrt(MSE)
print("Test RMSE : {}".format(np.round(RMSE,3)))